# Final Project

* In the United States, heart disease is the leading cause of death, accounting for one out of every four deaths. High blood pressure, high cholesterol, and smoking are all major risk factors for heart disease. Diabetes, obesity and overweight, physical inactivity, a poor diet, and excessive alcohol consumption are all medical problems and lifestyle choices that might increase the risk of heart disease. Heart disease risk can be considerably lowered by making lifestyle modifications and, in certain situations, taking medication.

The following are the goals of this project:
* Data on Missouri's leading causes of mortality from 2014 to 2017 is visualized and summarized.

* Evaluate Missouri residents' heart disease status at the county level, including heart disease mortality, heart disease hospitalizations, and emergency room visits, to see if there is a link between heart disease mortality and emergency room visits, average personal income, population density, and other variables.

* Examine whether there is a link between heart disease mortality and common risk factors in Missouri counties.

## 1. Data manipulation

#### Data Source

- Missouri leading causes of death information is retrieved from National center for health statistics in CDC website: https://www.cdc.gov/nchs/pressroom/states/missouri/missouri.htm.
- Heart disease profile data were downloaded as local files from Missouri Public Health Information Management System.
-  https://meric.mo.gov/data/income-measures/county-per-capita-personal-income, https://www.indexmundi.com/facts/united-states/quick-facts/missouri/land-area#table


#### Information about Missouri's major causes of mortality from 2014 to 2017.
The Missouri leading causes of death data were read from CDC website:https://www.cdc.gov/nchs/pressroom/states/missouri/missouri.htm
<br>


In [ ]:
# Read tables from website
import pandas as pd
import numpy as np
from pathlib import Path
tables = pd.read_html('https://www.cdc.gov/nchs/pressroom/states/missouri/missouri.htm')

In [ ]:
# 2014 -2017 Missouri leading causes of death data
data_2017 = tables[1]
data_2016 = tables[5]
data_2015 = tables[9]
data_2014 = tables[13]

In [ ]:
# function used to change columns names, assign a new colunm.

def changeColumns (dataframe, x):
    """(dataframe, str) -> dataframe
    
    This function returns a dataframe with the names of the columns changed and a new column added."""
    
    a = dataframe.columns[0][:-6]
    b = dataframe.columns[1]
    c = dataframe.columns[2][:4]
    d = dataframe.columns[3][:-1]
    e = dataframe.columns[4][:-2]
    dataframe.columns = [a, b, c, d, e]
    dataframe['Year'] = x
    dataframe[a] = dataframe[a].str[3:]
    dataframe.iloc[9,0] = dataframe.iloc[9,0][1:]
    
    dataframe = dataframe[[a,'Year',b,c,d,e]].copy()
    return dataframe

In [ ]:
# change data columns names
data_2017 = changeColumns(data_2017, 2017).copy()

In [ ]:
data_2016 = changeColumns(data_2016, 2016)

In [ ]:
data_2015 = changeColumns(data_2015, 2015)

In [ ]:
data_2014 = changeColumns(data_2014, 2014)

In [ ]:
# Merge 2014 - 2017 years' data
lcd_data = pd.concat([data_2014, data_2015, data_2016,data_2017])
lcd_data = lcd_data.reset_index()

In [ ]:
lcd_data.columns

In [ ]:
# long to wide pivoting tranformation
pdata = lcd_data.pivot(index='Year', columns='MO Leading Causes of Death', values=['Deaths','Rate'])

###  Data on heart disease death, hospitalization, and ER visits
View heart disease mortality, heart disease hospitalizations, and heart disease emergency room visits on a county level from local files. To obtain information about income and land area through website.
<br>
Death rates are age-adjusted and expressed as deaths per 100,000 of the population. The age-adjusted hospitalization and procedure rates are per 10,000 people, whereas the age-adjusted emergency department visit rates are per 1,000 people.

In [ ]:
# function to read csv and excel files

class InputModule:
    def __init__(self, fileFormat: str = "EXCEL"):
        self.fileFormat = fileFormat.upper()
        self.dataFrame = None
        self.filterList = ['Statewide', 'County']
        self.dropList = ['Lower 95% Confidence Limit', 'Upper 95% Confidence Limit', 'Significantly Different']

    def readfiles(self, filePath: Path):
        try:
            if self.fileFormat.upper() == "EXCEL":
                self.dataFrame = pd.read_excel(filePath, header=2, skipfooter=3)
            elif self.fileFormat.upper() == "CSV":
                self.dataFrame = pd.read_csv(filePath, header=2, skipfooter=3)
            else:
                print("Invalid File Format. Read Failure")
                SystemExit(1)
        except Exception as e:
            print("File Read Failure!!!, Filename:" + str(filePath) + " FileFormat:" + self.fileFormat)
            SystemExit(1)

        return self.dataFrame

    # function to filter data, drop columns, change columns names
    def formatDataFrame(self, pre: str, formatType: int = 1):
        self.dataFrame = self.dataFrame[self.dataFrame['Geography Type'].isin(self.filterList)]
        self.dataFrame = self.dataFrame.drop(
            self.dropList, axis=1)

        self.dataFrame.columns = [self.dataFrame.columns[0],
                                  self.dataFrame.columns[1],
                                  pre + '_' + self.dataFrame.columns[2] if type == 1 else self.dataFrame.columns[2],
                                  pre + '_' + self.dataFrame.columns[3],
                                  pre + '_' + self.dataFrame.columns[4]]

    def getDataFrame(self):
        return self.dataFrame


In [ ]:
# read heart disease data from local files.
    hd_data = InputModule('csv')
    hd_data.readfiles('coronary.csv')

In [ ]:
# heart disease mortality data
    hdm_data = InputModule('excel')
    hdm_data.readfiles('mortality_data.xlsx')


In [ ]:
# data for emergency room visits discharges with a principal diagnosis of heart disease
    erv_data = InputModule('excel')
    erv_data.readfiles('emergency_room_visits_data.xlsx')


In [ ]:
# data: hospitalizations discharges with a principal diagnosis of heart disease
    hd_discharge = InputModule('excel')
    hd_discharge.readfiles('hospitalization_data.xlsx')

In [ ]:
# transform data columns and filter data
    hd_data.formatDataFrame('Heart Disease')
    hdm_data.formatDataFrame('HD Mortality')
    erv_data.formatDataFrame('HD Emergency Room Visits')
    hd_discharge.formatDataFrame('HD Hospitalizations')

In [ ]:
from functools import reduce

In [ ]:
# Merge data
hdmc_data = reduce(lambda x,y: pd.merge(x,y, on=["Geography", "Geography Type"], how='left'), [hdm_data.getDataFrame(),
                                                                                                hd_data.getDataFrame(),
                                                                                                erv_data.getDataFrame(),
                                                                                                hd_discharge.getDataFrame()])

In [ ]:
#Retrieve personal income data from website
pcpi= pd.read_html('https://meric.mo.gov/data/income-measures/county-per-capita-personal-income', header = 1)
len(pcpi)

In [ ]:
pcpi1 = pcpi[1]

In [ ]:
# reshape dataframe
pi1 = pcpi1[pcpi1.columns[0:3]]
pi2 = pcpi1[pcpi1.columns[5:8]]
pi2.columns = pi1.columns
pi2['Area'] = pi2['Area'].replace(['St. Louis', 'St. Louis City'], ['St. Louis City', 'St. Louis County'])

In [ ]:
pi = pi1.append(pi2)
pi.columns = ['Geography', '2016 Per Capita Income', '2017 Per Capita Income']

In [ ]:
# Convert income data to numeric and append to heart disease mortality dataframe.
pi['2016 Per Capita Income'] = pi['2016 Per Capita Income'].str[1:].str.replace(',', '').astype(float)
pi['2017 Per Capita Income'] = pi['2017 Per Capita Income'].str[1:].str.replace(',', '').astype(float)
hdmc_data['2016 Per Capita Income'] = pi['2016 Per Capita Income'].values
hdmc_data['2017 Per Capita Income'] = pi['2017 Per Capita Income'].values

In [ ]:
# Retrieve county land area data
lands = pd.read_html('https://www.indexmundi.com/facts/united-states/quick-facts/missouri/land-area#table')
len(lands)

In [ ]:
# Manipulate land area data: change data columns name, add Missouri land area data to first row, sort data by county names.
land = lands[0]
land.columns= ['Geography', 'Area']
new_row = pd.DataFrame({'Geography':'Missouri', 'Area': 68741.52}, index = [0])
land = pd.concat([new_row, land]).reset_index(drop = True)
land.iloc[101,0] = 'St. Louis county'
land = land.sort_values(by ='Geography')

In [ ]:
hdmc_data.columns

####  Risk factors for heart disease
To read heart disease risk factors data from local files - data downloaded from Missouri Public Health Information Management System. The data include Risk Factors for Heart Disease Among Adults 18 Years and Older.
<br>


In [ ]:
# read adults 18 years and older with high blood pressure data by using readfiles function
high_bp = InputModule('csv')
high_bp.readfiles('blood pressure.csv')

In [ ]:
# read high cholesterol data
high_chol = InputModule('excel')
high_chol.readfiles('high_cholestrol_data.xlsx')

In [ ]:
# read current smoking data
smoking = InputModule('excel')
smoking.readfiles('current smoking.xlsx')

In [ ]:
# read diabetes data
diabetes = InputModule('excel')
diabetes.readfiles('diabetes_data.xlsx')

In [ ]:
# read obesity data
obesity = InputModule('excel')
obesity.readfiles('obesity_data.xlsx')

In [ ]:
# read overweight data
over_wt = InputModule('excel')
over_wt.readfiles('overweight_data.xlsx')

In [ ]:
# read physical inactivity data
ph_inactive = InputModule('excel')
ph_inactive.readfiles('physical_inactivity_data.xlsx')

In [ ]:
# transform data frames
high_bp.formatDataFrame('High Blood Pressure',2)
high_chol.formatDataFrame('High Cholesterol',2)
smoking.formatDataFrame('Smoking',2)
diabetes.formatDataFrame('Diabetes',2)
obesity.formatDataFrame('Obesity',2)
over_wt.formatDataFrame('Overweight',2)
ph_inactive.formatDataFrame('Physical Inactivity',2)

In [ ]:
ph_inactive.dataFrame.columns

In [ ]:
from functools import reduce

In [ ]:
# merge data
riskfactors = reduce(lambda x,y: pd.merge(x,y, on=["Geography", "Geography Type", "Data Years"], how='left'), [high_bp.getDataFrame(), high_chol.getDataFrame(), smoking.getDataFrame(), diabetes.getDataFrame(), obesity.getDataFrame(), over_wt.getDataFrame(), ph_inactive.getDataFrame()])

In [ ]:
rf1 = hdmc_data[['Geography', 'Geography Type','2016 Per Capita Income']]

In [ ]:
# Merge heart disease, heart disease mortality, population density, income data to risk factors data
hd_riskfactors = reduce(lambda x,y: pd.merge(x,y, on=["Geography", "Geography Type"], how='left'), [hd_data.getDataFrame(), hdm_data.getDataFrame(), riskfactors])

### 2. Data visualization

####  Missouri's major causes of mortality from 2014 to 2017

In [ ]:
# long to wide to pivot leading causes data to show the death rate
lcd_data.pivot(index='Year', columns='MO Leading Causes of Death', values='Rate')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.barplot(x='MO Leading Causes of Death', y='Rate', data=lcd_data)
plt.xticks(rotation=90)

* Interpretation: According to the table and graph, Heart disease is the top cause of death in Missouri, with a modest decrease in the incidence from 2014 to 2017.

####  Missouri resident heart disease mortality, hospitalizaitons, emergency room visits data and the relations.

In [ ]:
hdmc_data.head(2)

In [ ]:
# Calulate the maximum and minimum values and display the counties with maxium value or minium value
filter = hdmc_data['Geography Type'] == 'County'
df = hdmc_data[filter]
columnlist = [ 'HD Mortality_Rate','Heart Disease_Weighted Percent', \
          'HD Emergency Room Visits_Rate', 'HD Hospitalizations_Rate','2016 Per Capita Income']

In [ ]:
# Function to calulate the maximum and minimum values and display the counties with maxium value or minium value

def stat(df, columnlist):
    """(filepath, list) -> dataframe
    
    This function returns a new dataframe with statistic data."""
    sdata ={}
    for col in columnlist:
        ma = df[col].max()
        mal = df.loc[df[col] == df[col].max(), 'Geography'].values
        mi = df[col].min()
        mil = df.loc[df[col] == df[col].min(), 'Geography'].values
        me = df[col].mean()
        sdata[col] = {'County with maximum value': mal, 'Maximum value': ma, 'County with minimum value': mil, 'Minimum value': mi, \
                     'Average value': me}

    return(pd.DataFrame.from_dict(sdata))

In [ ]:
stat(df, columnlist)

In [ ]:
hdmc_data[columnlist].hist(figsize=(18,15))

In [ ]:
hdmc_data['2016 Per Capita Income/1000'] = hdmc_data['2016 Per Capita Income']/1000

In [ ]:
plist = hdmc_data[['HD Mortality_Rate','Heart Disease_Weighted Percent', 'HD Emergency Room Visits_Rate', \
          'HD Hospitalizations_Rate', '2016 Per Capita Income/1000']].copy()

In [ ]:
sns.pairplot(plist)

* Interpretation: According to the graphs, the incidence of heart disease death is linked to the rate of heart disease emergency department visits and heart disease hospitalization. The number of emergency department visits is linked to the number of inpatient discharges with a primary diagnosis of heart disease. Per capita income appears to be inversely associated with heart disease mortality in 2016.

In [ ]:
ax = sns.regplot(data=hdmc_data, x='HD Mortality_Rate', y='HD Emergency Room Visits_Rate', order=2, label='HD Emergency Room Visits_Rate')
sns.regplot(data=hdmc_data, x='HD Mortality_Rate', y='HD Hospitalizations_Rate', label='HD Hospitalizations_Rate', ax=ax)
ax.legend()
ax.set_ylabel('Rate')
ax.set_xlabel('Heart Disease Mortality_Rate')

In [ ]:
ax = sns.regplot(data=hdmc_data, x='HD Mortality_Rate', y='2016 Per Capita Income/1000', label='2016 Per Capita Income/1000')
ax.legend()
ax.set_ylabel('2016 Per Capita Income/1000')
ax.set_xlabel('Heart Disease Mortality_Rate')

#### Heart disease commom risk factors

In [ ]:
hd_riskfactors.columns

In [ ]:
# Calulate the maximum, minimum and average values and display the counties with maximum value or minimum value
filter = hd_riskfactors['Geography Type'] == 'County'
df2 = hd_riskfactors[filter]
columnlist2 = [ 'HD Mortality_Rate','Heart Disease_Weighted Percent', 'High Blood Pressure_Weighted Percent',\
               'High Cholesterol_Weighted Percent','Smoking_Weighted Percent','Diabetes_Weighted Percent','Obesity_Weighted Percent', \
               'Overweight_Weighted Percent','Physical Inactivity_Weighted Percent']

In [ ]:
stat(df2, columnlist2)

In [ ]:
clists = columnlist2[2:]
for i in clists:
    ax = sns.regplot(data=hd_riskfactors, x='HD Mortality_Rate', y=i, label=i)
    ax.legend()
    ax.set_ylabel('Risk Factors')
    ax.set_xlabel('Heart Disease Mortality_Rate')
    plt.show()

In [ ]:
clists = columnlist2[2:]
for i in clists:
    ax = sns.lmplot(data=hd_riskfactors, x="Heart Disease_Weighted Percent", y=i)
    plt.show()

In [ ]:
for i in columnlist2[3:7]:
    ax = sns.regplot(data=hd_riskfactors, x='High Blood Pressure_Weighted Percent', y=i, label=i)
    ax.legend()
    ax.set_ylabel('Risk Factors')
    ax.set_xlabel('High Blood Pressure_Weighted Percent')
    plt.show()

* Interpretation: It appears that current smoking, obesity, overweight, and physical inactivity have a positive connection with heart disease mortality. Heart disease and average personal income have negative connections.

### 3. Correlation Analysis

In [ ]:
from scipy.stats import pearsonr

In [ ]:
def correlations(dt1, dt2, columnlists):
    """(dataframe, dataframe, list) -> print
    
    This function calculate and print the coefficent and p-value."""
    for i in columnlists:
        a = pearsonr(dt1, dt2[i])
        print(dt1.name+', '+ i +': coefficent: {}, p-value: {}'.format(round(a[0], 5), round(a[1],5)))

In [ ]:
correlations(hdmc_data['HD Mortality_Rate'], hdmc_data, columnlist[2:])

* Interpretation: Because the sample sizes are limited, certain correlation coefficients are not significant. Other state data could be added in the future to increase sample sizes.

---



### Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add Final.ipynb
    !git commit -a -m "Submitting the Final programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')